In [29]:
import numpy as np
import pandas as pd
import math
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance

In [30]:
train = pd.read_csv('dataset/development_dataset.csv')
test = pd.read_csv('dataset/leaderboard_dataset.csv')

In [31]:
train['VAR14'] = train['VAR14'].replace('.',1)
train['VAR21'] = train['VAR21'].replace({'Low','Medium','High'},(0,1,2))

In [32]:
train['VAR14']=train['VAR14'].astype(str).astype(int)
train['VAR21']=train['VAR21'].astype(str).astype(int)
test['VAR14'] = test['VAR14'].replace('.',1)
test['VAR14']=test['VAR14'].astype(str).astype(int)

In [33]:
train['VARS1'] = abs(train['VAR7'] - train['VAR16'])
test['VARS1'] = abs(test['VAR7'] - test['VAR16'])

In [34]:
x = train.drop(['VAR21'],axis=1)
y = train['VAR21']

In [35]:
columns = ['VAR15','VAR18','VAR7','VAR1']
xc = x.drop(columns, axis=1)
testc = test.drop(columns,axis=1)

In [36]:
D_train = xgb.DMatrix(xc, label=y)
D_test = xgb.DMatrix(testc)

In [37]:
param = {
    'eta': 0.1, 
    'max_depth': 9,  
    'objective': 'multi:softmax',
    'min child wt' :7,
    'gamma': 5,
    'num_class': 3} 

In [38]:
model = xgb.train(param,D_train, 100)

In [39]:
prediction = model.predict(D_test)

In [40]:
output = pd.DataFrame()
output['VAR1'] = test['VAR1']
output['VAR21']= prediction

In [41]:
output['VAR21'].value_counts()

0.0    7607
2.0    2278
1.0     115
Name: VAR21, dtype: int64

In [42]:
output['VAR21'] = output['VAR21'].replace((0,1,2),{'Low','Medium','High'})

In [43]:
output.to_csv(r'Neil_Hanna_IITKGP.csv', header = None, index=False)

In [44]:
param1 = {
    'eta': 0.1, 
    'max_depth': 9,  
    'objective': 'multi:softprob',
    'min child wt' :7,
    'gamma': 5,
    'num_class': 3} 

In [45]:
model1 = xgb.train(param1,D_train, 100)

In [46]:
pred1 =  model1.predict(D_test)

In [47]:
model2 = xgb.train(param1,D_train, 150)

In [48]:
pred2 =  model2.predict(D_test)

In [49]:
model3 = xgb.train(param1,D_train, 80)

In [50]:
pred3 =  model3.predict(D_test)

In [51]:
res = np.zeros(pred1.shape)

In [52]:
for i in range(0,10000):
    res[i] = 0.8*pred2[i]+0.4*pred3[i]+0.3*pred1[i]
out = pd.DataFrame(res)

In [53]:
output = pd.DataFrame()
output['VAR1'] = test['VAR1']
output['VAR21']= out.idxmax(axis=1)

In [54]:
output['VAR21'].value_counts()

0    7606
2    2279
1     115
Name: VAR21, dtype: int64

In [55]:
output['VAR21'] = output['VAR21'].replace((0,1,2),{'Low','Medium','High'})
output.to_csv(r'Neil_hanna_IITKGP.csv', header = None, index=False)